In [ ]:
import math

#===================================================================================================
# INPUTS
#===================================================================================================

K = 8.314472 * 0.001  # Gas constant in kJ/mol/K    kJ/mol/K
V = 1660              # average volume per molecule for a reference concentration of  is 1.661 x 10^3 Å^3/molecule.

T      = 300.0          # Temperature in Kelvin

r0      = 0       # Distance in A
thA_0   = 0     # Angle in degrees (protein)
thB_0   = 0     # Angle in degrees (ligand)

#  kcal * 4.186 ---->  kJ
#
# AMBER restraint = k * d_x ^ 2, boresch u = 1/2 * k * d_x ^ 2
# 
# BORESCH FORMULA
K_r    = 2 * 20 * 4.186      # force constant for distance (kJ/mol/A^2)  
K_thA  = 2 * 200 * 4.186     # force constant for angle    (kJ/mol/rad^2)
K_thB  = 2 * 200 * 4.186     # force constant for angle    (kJ/mol/rad^2)
K_phiA = 2 * 200 * 4.186     # force constant for dihedral (kJ/mol/rad^2)
K_phiB = 2 * 200 * 4.186     # force constant for dihedral (kJ/mol/rad^2)
K_phiC = 2 * 200 * 4.186     # force constant for dihedral (kJ/mol/rad^2)

# ROUX FORMULA
K_dist    = 2 * 20 * 4.186     # force constant for distance    (kJ/mol/A^2)  
K_ang     = 2 * 200 * 4.186    # force constant for angle       (kJ/mol/rad^2)
K_rotate  = 2 * 200 * 4.186    # force constant for dihedral    (kJ/mol/rad^2)

# AMBER RESTRAINT CONSTANT
K_DIST      = 20           # force constant for distance    (kcal/mol/A^2)  
K_ANG       = 200          # force constant for angle       (kcal/mol/rad^2)
K_ROTATE    = 200          # force constant for dihedral    (kcal/mol/rad^2)

In [ ]:
with open('./init_structure/Boresch_restraint.tmpl','r') as f:
    lines = [line for line in f.readlines() if line.strip()]
    r0 = float(lines[1].split()[1].split('=')[1].strip(','))
    thA_0 = float(lines[5].split()[1].split('=')[1].strip(','))
    thB_0 = float(lines[9].split()[1].split('=')[1].strip(','))
    dihedral_P_0 = float(lines[13].split()[1].split('=')[1].strip(','))
    dihedral_M_0 = float(lines[17].split()[1].split('=')[1].strip(','))
    dihedral_L_0 = float(lines[21].split()[1].split('=')[1].strip(','))

In [ ]:
#===================================================================================================
# BORESCH FORMULA
#===================================================================================================

thA = math.radians(thA_0)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB_0)  # convert angle from degrees to radians --> math.sin() wants radians

arg =(
    (8.0 * math.pi**2.0 * V) / (r0**2.0 * math.sin(thA) * math.sin(thB)) 
    * 
    (
        ( (K_r * K_thA * K_thB * K_phiA * K_phiB * K_phiC)**0.5 ) / ( (2.0 * math.pi * K * T)**(3.0) )
    )
)

In [ ]:
dG = - K * T * math.log(arg)

print ("dG_off = %8.3f kcal/mol" %(dG/4.186))
print ("dG_on  = %8.3f kcal/mol" %(-dG/4.186))
# 1 kcal = 4.186 KJ

In [ ]:
#===================================================================================================
# ROUX FORMULA
#===================================================================================================

thA = math.radians(thA_0)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB_0)  # convert angle from degrees to radians --> math.sin() wants radians

Ft = r0 ** 2 * math.sin(thA) * ((2.0 * math.pi * K * T)**(3.0) / (K_dist * K_ang ** 2)) ** 0.5

Fr = math.sin(thB) * (2.0 * math.pi * K * T / K_rotate) ** 1.5 / (8 * math.pi ** 2)

In [ ]:
dG = - K * T * math.log(Ft / V * Fr)
print (f"dG = {dG/4.186:.3f} kcal/mol")

In [ ]:
data_dir = './complex_1_20_200_200/extract_6_DOFs'
restraints = ['Distance', 'Angle_P', 'Angle_L', 'Dihedral_P','Dihedral_M','Dihedral_L']

In [ ]:
import numpy as np

In [ ]:
distance = np.loadtxt(f'{data_dir}/{restraints[0]}.dat',usecols=1,dtype=float)
# mbar state 1.0
mabr_distance = K_DIST * np.power(distance-r0,2)


In [ ]:
mabr_distance[:10]

In [ ]:
angle_p = np.loadtxt(f'{data_dir}/{restraints[1]}.dat',usecols=1,dtype=float)
angle_L = np.loadtxt(f'{data_dir}/{restraints[2]}.dat',usecols=1,dtype=float)

In [ ]:
mbar_angle = K_ANG * (
    np.power(np.radians(angle_p-thA_0),2) +
    np.power(np.radians(angle_L-thB_0),2) 
    )

In [ ]:
dihedral_P = np.loadtxt(f'{data_dir}/{restraints[3]}.dat',usecols=1,dtype=float)
dihedral_M = np.loadtxt(f'{data_dir}/{restraints[4]}.dat',usecols=1,dtype=float)
dihedral_L = np.loadtxt(f'{data_dir}/{restraints[5]}.dat',usecols=1,dtype=float)


In [ ]:
mbar_dihedral = K_ROTATE * (
    np.radians(dihedral_P - dihedral_P_0) ** 2 +
    np.radians(dihedral_M - dihedral_M_0) ** 2 +
    np.radians(dihedral_L - dihedral_L_0) ** 2
)

In [ ]:
mbar_state_1_0 = mbar_dihedral + mbar_angle + mabr_distance

In [ ]:
states = [ 0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.35, 0.5, 0.75, 1.0]

In [ ]:
beta = 1.0 / (300 * 1.380649e-23 )
# (RESTRAINT * 4.186 * 1000 / 6.02214076e23) * beta
u_kn = [mbar_state_1_0 * 4.186 * 1000 / 6.02214076e23 * beta * state for state in states]

In [ ]:
mbar_energy = np.array(u_kn)

In [ ]:
N_k = np.full(len(states),1000,dtype=np.int32)

In [ ]:
from pymbar import MBAR

In [ ]:
mbar = MBAR(u_kn,N_k)

In [ ]:
Delta_f_ij, dDelta_f_ij, Theta_ij = mbar.getFreeEnergyDifferences(return_theta=True)

In [ ]:
dDelta_f_ij

In [ ]:
print('Delta_f_ij.shape:',Delta_f_ij.shape)
print(f"Restraint energy : {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol")
with open(f'{data_dir}/Restraint_energy.txt','w') as f:
    f.write(f'Restraint energy(MBAR): {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol\n')
    for i in range(11):
        f.writelines(f'{Delta_f_ij[i][i+1] * 0.596:.3f}\n')

In [ ]:
import pandas as pd
pd.DataFrame(Delta_f_ij * 0.596)

In [ ]:
O_ij = mbar.computeOverlap()['matrix']
np.savetxt(f'{data_dir}/restraint_overlap.txt',O_ij,fmt='%.8f')
O_ij.round(decimals=3, out=O_ij)
pd.DataFrame(O_ij)